In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

MessageError: ignored

In [ ]:
import os

#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


import torch, torchvision
from torch import nn
from pathlib import Path
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.transforms import RandomCrop, RandomHorizontalFlip, RandomRotation
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from tqdm import tqdm
from typing import List, Tuple, Dict
from skimage import io, transform, color
from torchvision.transforms import functional as F

!pip install elasticdeform
import elasticdeform

import random

import cv2

from PIL import Image

import numpy as np

import multiprocessing as mp


#### To unlock to check the integrity of the dataset. Are there differences between images and masks?
"""
def checkDataset(img_path, mask_path):
  file_set1 = set()
  for dirpath, dirnames, filenames in os.walk(img_path):
    for filename in filenames:
      file_name, file_extension = os.path.splitext(filename)
      file_set1.add(file_name)

  file_set2 = set()
  for dirpath, dirnames, filenames in os.walk(mask_path):
    for filename in filenames:
      file_name, file_extension = os.path.splitext(filename)
      file_set2.add(file_name)

  # compare the sets of file names
  unique_to_folder1 = file_set1 - file_set2
  unique_to_folder2 = file_set2 - file_set1


  # print the results
  print("Files unique to folder 1:", unique_to_folder1)
  print("Files unique to folder 2:", unique_to_folder2)

"""


# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"


def walk_through_dir(dir_path):
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

data_path = Path("/content/gdrive/MyDrive/FoodSeg103/Images/")

img_dir = data_path / "img_dir"
ann_dir = data_path / "ann_dir"


# RandomCustomCrop
class RandomCustomCrop(object):

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):

        img, mask = sample['image'], sample['mask']

        w = img.shape[0]
        h = img.shape[1]

        #print("w", w)
        #print("h", h)

        # New crop size passed in the constructor
        new_w, new_h = self.output_size

        if h == new_h and w == new_w:
            return sample

        if w < new_w or h < new_h:
            raise ValueError("Input image size is smaller than the desired crop size")
         #   #return sample

        x1 = random.randint(0, w - new_w)
        y1 = random.randint(0, h - new_h)

        # Convert to PIL Image
        img = Image.fromarray(img)
        mask = Image.fromarray(mask)

        # Crop img and mask
        img = F.crop(img, y1, x1, new_h, new_w)
        mask = F.crop(mask, y1, x1, new_h, new_w)

        # PIL to numpy array
        img = np.array(img)
        mask = np.array(mask)

        return {'image': img, 'mask': mask}


# RandomCustomRotation
class RandomCustomRotation(object):


    def __init__(self, degrees):
        self.degrees = degrees

    def __call__(self, sample):

        img, mask = sample['image'], sample['mask']

        angle = random.uniform(self.degrees[0], self.degrees[1])

        # Convert to PIL Image
        img = Image.fromarray(img)
        mask = Image.fromarray(mask)

        # Rotation
        img = F.rotate(img, angle)
        mask = F.rotate(mask, angle)

        # PIL to numpy array
        img = np.array(img)
        mask = np.array(mask)

        return {'image': img, 'mask': mask}


# RandomCustomHorizontalFlip
class RandomCustomHorizontalFlip(object):
    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, sample):
        img, mask = sample['image'], sample['mask']



        if random.random() < self.p:

            # Transformation to tensors
            img = torch.from_numpy(img).float()
            mask = torch.from_numpy(mask).float()

            img = torch.flip(img, dims=[1])
            mask = torch.flip(mask, dims=[1])

            # Transform back
            img = img.numpy()
            mask = mask.numpy()

        return {'image': img, 'mask': mask}


# RandomCustomVerticalFlip
class RandomCustomVerticalFlip(object):
    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, sample):
        img, mask = sample['image'], sample['mask']


        if random.random() < self.p:

            # Transformation to tensors
            img = torch.from_numpy(img).float()
            mask = torch.from_numpy(mask).float()

            img = torch.flip(img, dims=[0])
            mask = torch.flip(mask, dims=[0])

            # Transform back
            img = img.numpy()
            mask = mask.numpy()

        return {'image': img, 'mask': mask}



class RandomGaussianNoise(object):
    def __init__(self, mean=0.0, std=0.1):
      self.mean = mean
      self.std  = std

    def __call__(self, img):

      # Generate Gaussian noise with mean 0 and standard deviation 10 for each color channel
      noise = np.random.normal(0, 1, img.shape)

      # Scale the noise by a small factor (e.g., 0.05) to add a light layer of noise
      scaled_noise = 0.1 * noise

      # Add the noise to each color channel of the image
      noisy_img = np.clip(img.astype(np.int16) + scaled_noise, 0, 255).astype(np.uint8)

      img = noisy_img

      # Transform back
      #img = img.numpy()


      return img


# ElasticDeform
class ElasticDeform(object):
    def __init__(self, sigma, alpha):
        self.sigma = sigma
        self.alpha = alpha

    def __call__(self, sample):
        img, mask = sample['image'], sample['mask']


        # We add a dimension at the end
        mask = np.expand_dims(mask, axis=-1)



        # Ensure that the input image is a 3D array with shape (height, width, channels)
        assert img.ndim == 3 and img.shape[2] == 3, "Input image must be a 3D RGB array with shape (height, width, 3)"

        # Ensure that the mask is a 3D array with shape (height, width, 1)
        assert mask.ndim == 3 and mask.shape[2] == 1, "Mask must be a 3D array with shape (height, width, 1)"

        # Reshape the input image into a 2D array with shape (height * width, channels)
        height, width, channels = img.shape
        flattened_img = img.reshape(-1)

        mask_rgb = np.repeat(mask, 3, axis=-1)
        flattened_mask = mask_rgb.reshape(-1)


        # Generate a random displacement field along the width and height dimensions
        #displacement = np.random.randn(2, height, width) * self.sigma

        # Combine images and masks into a single 3D array
        combined = np.concatenate((flattened_img, flattened_mask), axis=-1)

        # Apply deformation to combined array
        deformed_combined = elasticdeform.deform_random_grid(combined,
                                                     sigma=self.sigma,
                                                     order=3,
                                                     mode='nearest'
                                                     )

        # Split deformed array back into images and masks
        deformed_reshaped_img = deformed_combined[:height*width*channels].reshape((height, width, channels))
        deformed_reshaped_mask = deformed_combined[height*width*channels:].reshape((height, width, channels))

        #Convert mask to grayscale
        deformed_reshaped_mask = np.mean(deformed_reshaped_mask, axis=-1)
        deformed_reshaped_mask = np.expand_dims(deformed_reshaped_mask, axis=-1)

        #deformed_mask = np.squeeze(deformed_reshaped_mask, axis=-1)

        deformed_img = deformed_reshaped_img
        deformed_mask = deformed_reshaped_mask

        print("deformed_img", deformed_img.shape)
        print("deformed_mask", deformed_mask.shape)

        return {'image': deformed_img, 'mask': deformed_mask}




class CustomToTensor(object):
    #Convert ndarrays in sample to Tensors.


    def __new__(self, sample):
        img, mask = sample['image'], sample['mask']

        mask = np.expand_dims(mask, axis=-1)

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        img = img.transpose((2, 0, 1))
        mask = mask.transpose((2, 0, 1))

        return {'image': torch.from_numpy(img).float(),
                'mask': torch.from_numpy(mask).float()}


class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor


# Class SegIngredientDataset
class SegIngredientsDataset(Dataset):

    def __init__(self, img_dir, mask_dir, path, img_transform, mask_transform, common_transform, eval):

        self.eval = eval

        self.img_dir = img_dir / path
        self.mask_dir = mask_dir / path

        self.img_transform = img_transform
        self.mask_transform = mask_transform
        self.common_transform = common_transform

        print(self.img_dir)

        # Traverses and prints directory details
        #walk_through_dir(img_dir)

        #walk_through_dir(mask_dir)

        self.img_list = []
        self.mask_list = []

        self.dirs = [self.img_dir, self.mask_dir]

        self.num_processes = min(mp.cpu_count(), len(self.dirs))

    # Return the instance
    def Get(self):
        return self

    # Method called by the processs to do parallelization
    def run(self, obj):
        pool = mp.Pool(self.num_processes)

        pool.daemon = False

        manager = mp.Manager()
        p_img_list = manager.list()
        p_mask_list = manager.list()

        pool.starmap(self.read_files_from_folder, [(self.img_dir, p_img_list, cv2.IMREAD_COLOR), (self.mask_dir, p_mask_list, cv2.IMREAD_GRAYSCALE)])

        self.img_list = p_img_list
        self.mask_list = p_mask_list

        pool.close()
        pool.join()

    # Subprocess reading in the 2 different folders at the same time
    def read_files_from_folder(self, folder_path, custom_list, imread_type):
        counter = 0
        for entry in sorted(os.scandir(folder_path), key=lambda e: e.name):
            if entry.is_file():
              img_path = entry.path

              img_open = Image.open(img_path)
              img_open = img_open.resize((256, 256), resample=Image.NEAREST)

              img_array = np.array(img_open)
              custom_list.append(img_array)
              #print(filename, " | ", np.unique(img_array))
              print(img_path)


            #counter = counter + 1
            #if(counter >17):
            #  break


    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):

        img = self.img_list[idx]
        y = self.mask_list[idx]


        # Create sample array to perform modifications simultaneously on both images and masks
        sample = {}
        sample['image'] = img
        sample['mask'] = y



        if(self.eval):
          sample_transformed = CustomToTensor(sample)
          img = sample_transformed['image']
          y = sample_transformed['mask']


          return img, y

        if self.img_transform and self.mask_transform and self.common_transform is not None:

            # Perform modifications together
            sample_transformed = self.common_transform(sample)
            img = sample_transformed['image']
            y = sample_transformed['mask']

            img = self.img_transform(img)
            y = self.mask_transform(y)


        return img, y


# Parallelization

from multiprocessing import Process, Manager
from multiprocessing.managers import BaseManager

BaseManager.register('SegIngredientsDataset', SegIngredientsDataset)
manager = BaseManager()
manager.start()

#####
#Transformations
#########


#Normalization image

img_mean = [0.485, 0.456, 0.406]
img_std = [0.229, 0.224, 0.225]


#RandomGaussianNoise(mean=0.0, std=1.0),
img_transform = transforms.Compose([

    transforms.ToTensor(),
    transforms.Normalize(img_mean, img_std)
])

# Mask transform
mask_transform = transforms.Compose([
    transforms.ToTensor(),
])

# Common transform
#RandomCustomCrop((200,200)),
#RandomCustomRotation(degrees=(-50, 50)),

#RandomCustomVerticalFlip(),
#ElasticDeform(sigma=0.1, alpha=30),
common_transform = transforms.Compose([

    RandomCustomRotation(degrees=(-50, 50)),
    RandomCustomHorizontalFlip(),

])


# Classes handled by the process manager

train_data = manager.SegIngredientsDataset(img_dir = img_dir, mask_dir = ann_dir, path = "train/", img_transform=img_transform, mask_transform=mask_transform, common_transform=common_transform, eval=False)
test_data  = manager.SegIngredientsDataset(img_dir = img_dir, mask_dir = ann_dir, path = "test/", img_transform=img_transform, mask_transform=mask_transform, common_transform=common_transform, eval=True)

process1 = mp.Process(target=train_data.run, args=[train_data])
process2 = mp.Process(target=test_data.run, args=[test_data])
process1.start()
process2.start()

process1.join()
process2.join()

# Retrieve the 2 datasets (train and test)
train_1 = train_data.Get()
test_1 = test_data.Get()


Streaming output truncated to the last 5000 lines.
/content/gdrive/MyDrive/FoodSeg103/Images/img_dir/train/00002374.jpg
/content/gdrive/MyDrive/FoodSeg103/Images/ann_dir/train/00002600.png
/content/gdrive/MyDrive/FoodSeg103/Images/ann_dir/train/00002601.png
/content/gdrive/MyDrive/FoodSeg103/Images/img_dir/train/00002375.jpg
/content/gdrive/MyDrive/FoodSeg103/Images/ann_dir/train/00002602.png
/content/gdrive/MyDrive/FoodSeg103/Images/img_dir/train/00002376.jpg
/content/gdrive/MyDrive/FoodSeg103/Images/ann_dir/train/00002603.png
/content/gdrive/MyDrive/FoodSeg103/Images/img_dir/train/00002377.jpg
/content/gdrive/MyDrive/FoodSeg103/Images/ann_dir/train/00002604.png
/content/gdrive/MyDrive/FoodSeg103/Images/img_dir/train/00002378.jpg
/content/gdrive/MyDrive/FoodSeg103/Images/ann_dir/train/00002605.png
/content/gdrive/MyDrive/FoodSeg103/Images/img_dir/train/00002379.jpg
/content/gdrive/MyDrive/FoodSeg103/Images/ann_dir/train/00002606.png
/content/gdrive/MyDrive/FoodSeg103/Images/ann_dir/tr

In [ ]:
# Calculate pixel accuracy
def calculate_pixel_accuracy(y_truth, y_pred):

    matched_pixels = (y_truth == y_pred).sum().item()
    total_pixels = y_truth.nelement()
    #print("matched pixels", matched_pixels)
    match_ = (matched_pixels / total_pixels)

    return match_



In [ ]:
# We create a train_loader by passing the train dataset
train_loader = torch.utils.data.DataLoader(dataset = train_1,
                                            batch_size = 4,
                                            shuffle = True,
                                            drop_last=True) #shuffle ensures we traverse images in different order across epochs


# Display image and label. Checking if everything is fine
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

# Remove extra dimension (batch)
img = train_features[0].squeeze()
label = train_labels[0].squeeze()
print(img.size())

#print(label)


# Unnormalize the data so it appears clearly:

img_unnorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
#mask_unnorm = UnNormalize(mean=(0.5,), std=(0.5,))

img_new = img_unnorm(img)
#mask_unnorm = mask_unnorm(label)

img_new = img_new.permute(1,2,0)

#print(img_new)





# normalize the tensor values to the range [0, 255]
img_new = img_new * 255.0

# convert the tensor to a numpy array
img_new = img_new.clamp(0, 255).byte().numpy()

#print(img_new)

"""
Label

"""

# normalize the tensor values to the range [0, 255]
label_new = label * 255.0

# convert the tensor to a numpy array
label_new = label_new.clamp(0, 255).byte().numpy()

#print(label_new)


plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Train Image')
plt.imshow(img_new)
plt.subplot(232)
plt.title('Train Label/Mask')
plt.imshow(label_new, cmap="gray")

plt.show()


del train_loader


Feature batch shape: torch.Size([4, 3, 256, 256])
Labels batch shape: torch.Size([4, 1, 256, 256])
torch.Size([3, 256, 256])


In [ ]:
#if not FIRST_RUN:
#  del train_step

def train_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module,
optimizer: torch.optim.Optimizer):
  total_train = 0
  correct_train = 0
# Put model in train mode
  model.train()
# Setup train loss and train accuracy values
  train_loss, train_acc, train_miou_acc_all, train_dice_acc, train_miou_acc_ibg, train_iou_background = 0, 0, 0, 0, 0, 0
# Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):

    y = y.squeeze(dim=1)
    y = y.type(torch.LongTensor)
    #X = X.float()

    #print("y", y.shape )


# Send data to target device
    X, y = X.to(device), y.to(device)
# 1. Forward pass
    y_pred = model(X)
    #print("y pred shape", y_pred.shape)
# 2. Calculate and accumulate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
# 3. Optimizer zero grad
    optimizer.zero_grad()
# 4. Loss backward
    loss.backward()
# 5. Optimizer step
    optimizer.step()
# Calculate and accumulate accuracy metric across all batches

    predicted = torch.argmax(y_pred, 1)

    train_acc += calculate_pixel_accuracy(y, predicted)

    train_tp, train_fp, train_fn, train_tn = smp.metrics.get_stats(predicted, y, mode='multiclass', num_classes=104)
    train_miou_acc_all += smp.metrics.iou_score(train_tp, train_fp, train_fn, train_tn, reduction="micro").item()
    train_dice_acc += smp.metrics.f1_score(train_tp, train_fp, train_fn, train_tn, reduction="micro").item()

    predicted_i = predicted.clone() - 1
    y_i = y.clone() - 1

    train_tp_i, train_fp_i, train_fn_i, train_tn_i = smp.metrics.get_stats(predicted_i, y_i, mode='multiclass', num_classes=104, ignore_index=-1)
    train_miou_acc_ibg += smp.metrics.iou_score(train_tp_i, train_fp_i, train_fn_i, train_tn_i, reduction="micro").item()




# Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)

  train_acc = train_acc / len(dataloader)

  train_miou_acc_all = train_miou_acc_all / len(dataloader)
  train_dice_acc = train_dice_acc / len(dataloader)

  train_miou_acc_ibg = train_miou_acc_ibg / len(dataloader)

  train_iou_background = train_miou_acc_all - train_miou_acc_ibg


  return train_loss, train_acc, train_miou_acc_all, train_dice_acc, train_miou_acc_ibg, train_iou_background


In [ ]:
#if not FIRST_RUN:
#  del test_step

def test_step(model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module):

  total_test = 0
  correct_test = 0
# Put model in eval mode
  model.eval()
# Setup test loss and test accuracy values
  test_loss, test_acc, test_miou_acc_all, test_dice_acc, test_miou_acc_ibg, test_iou_background = 0, 0, 0, 0, 0, 0
# Turn on no_grad context manager
  with torch.no_grad():
# Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):

      y = y.squeeze(dim=1)
      y = y.type(torch.LongTensor)

# Send data to target device
      X, y = X.to(device), y.to(device)
# 1. Forward pass
      test_pred_logits = model(X)
# 2. Calculate and accumulate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()
# Calculate and accumulate accuracy

      predicted = torch.argmax(test_pred_logits, 1)

      test_acc += calculate_pixel_accuracy(y, predicted)

      test_tp, test_fp, test_fn, test_tn = smp.metrics.get_stats(predicted, y, mode='multiclass', num_classes=104)
      test_miou_acc_all += smp.metrics.iou_score(test_tp, test_fp, test_fn, test_tn, reduction="micro").item()
      test_dice_acc += smp.metrics.f1_score(test_tp, test_fp, test_fn, test_tn, reduction="micro").item()

      predicted_i = predicted.clone() - 1
      y_i = y.clone() - 1

      test_tp_i, test_fp_i, test_fn_i, test_tn_i = smp.metrics.get_stats(predicted_i, y_i, mode='multiclass', num_classes=104, ignore_index=-1)
      test_miou_acc_ibg += smp.metrics.iou_score(test_tp_i, test_fp_i, test_fn_i, test_tn_i, reduction="micro").item()




# Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)

  test_acc = test_acc / len(dataloader)

  test_miou_acc_all = test_miou_acc_all / len(dataloader)
  test_dice_acc = test_dice_acc / len(dataloader)

  test_miou_acc_ibg = test_miou_acc_ibg / len(dataloader)

  test_iou_background = test_miou_acc_all - test_miou_acc_ibg

  return test_loss, test_acc, test_miou_acc_all, test_dice_acc, test_miou_acc_ibg, test_iou_background


In [ ]:
#if not FIRST_RUN:
#  del save_training_checkpoint
#  del save_results_in_file
#  del train

import json

# Function to save pth files to google drive
def save_training_checkpoint(state, filename):
    path = F"/content/gdrive/MyDrive/Thesis_deeplearning_models/{filename}.pth"
    torch.save(state, path)

# Function to save results into a file
def save_results_in_file(dic_results, hyperparameters):
  link = "/content/gdrive/MyDrive/Thesis_deeplearning_models/results_bo_v" + str(hyperparameters.VERSION_EXPERIMENT) + ".txt"
  with open(link, 'a') as f:
    json.dump(dic_results, f)
    f.write('\n')


# Define a custom encoder that skips over non-serializable attributes
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, property):
            return None
        return super().default(obj)


def save_hyperparameters_in_file(hyperparameters):
  # Convert the class variables to a dictionary
  table_dict = {"VERSION_EXPERIMENT": hyperparameters.VERSION_EXPERIMENT,
                "OPTIMIZER": hyperparameters.OPTIMIZER,
                "ARCHITECTURE_USED" : hyperparameters.ARCHITECTURE_USED,
                "NETWORK": hyperparameters.NETWORK,
                "BATCH_SIZE": hyperparameters.BATCH_SIZE,
                "LEARNING_RATE": hyperparameters.LEARNING_RATE,
                "MOMENTUM": hyperparameters.MOMENTUM,
                "WEIGHT_DECAY": hyperparameters.WEIGHT_DECAY,
                "LOSS_FUNCTION" : hyperparameters.LOSS_FUNCTION,
                "NUM_EPOCHS": hyperparameters.NUM_EPOCHS
                }

  # Serialize the dictionary to a JSON string
  json_string = json.dumps(table_dict, indent=4)
  json_string_comma = "," + json_string
  link = "/content/gdrive/MyDrive/Thesis_deeplearning_models/hyperparameters_versions.txt"

  # Write the JSON string to a text file
  with open(link, "a") as f:
      f.write(json_string_comma)



def train(model: torch.nn.Module, train_dataloader: torch.utils.data.DataLoader, test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer, hyperparameters, loss_fn: torch.nn.Module = nn.CrossEntropyLoss(), epochs: int = 5):
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "train_miou_acc_all": [],
        "train_dice_acc" : [],
        "train_miou_acc_ibg" : [],
        "train_iou_background": [],
        "test_loss": [],
        "test_acc": [],
        "test_miou_acc_all": [],
        "test_dice_acc": [],
        "test_miou_acc_ibg" : [],
        "test_iou_background": [],
        "learning_rate": [],
    }

    results_line = {
        "epoch": 0,
        "train_loss": 0,
        "train_acc": 0,
        "train_miou_acc_all": 0,
        "train_dice_acc" : 0,
        "train_miou_acc_ibg" : 0,
        "train_iou_background": 0,
        "test_loss": 0,
        "test_acc": 0,
        "test_miou_acc_all": 0,
        "test_dice_acc": 0,
        "test_miou_acc_ibg" : 0,
        "test_iou_background": 0,
        "learning_rate": 0,
    }

    # Save hyperparameters in a file
    save_hyperparameters_in_file(hyperparameters)

    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc, train_miou_acc_all, train_dice_acc, train_miou_acc_ibg, train_iou_background = train_step(model=model,dataloader=train_dataloader, loss_fn=loss_fn, optimizer=optimizer)
        test_loss, test_acc, test_miou_acc_all, test_dice_acc, test_miou_acc_ibg, test_iou_background = test_step(model=model, dataloader=test_dataloader, loss_fn=loss_fn)

        #current_lr = scheduler.get_last_lr()[0]
        #scheduler.step()
        current_lr = hyperparameters.LEARNING_RATE


        # 4. Print loss & accuracies
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"train_miou_acc_all: {train_miou_acc_all:.4f} | "
            f"train_dice_acc: {train_dice_acc:.4f} | "
            f"train_miou_acc_ibg: {train_miou_acc_ibg:.4f} | "
            f"train_iou_background: {train_iou_background:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f} | "
            f"test_miou_acc_all: {test_miou_acc_all:.4f} | "
            f"test_dice_acc: {test_dice_acc:.4f} | "
            f"test_miou_acc_ibg: {test_miou_acc_ibg:.4f} | "
            f"test_iou_background: {test_iou_background:.4f} | "
            f"learning_rate: {current_lr:.4f} | "
        )
        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["train_miou_acc_all"].append(train_miou_acc_all)
        results["train_dice_acc"].append(train_dice_acc)
        results["train_miou_acc_ibg"].append(train_miou_acc_ibg)
        results["train_iou_background"].append(train_iou_background)

        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        results["test_miou_acc_all"].append(test_miou_acc_all)
        results["test_dice_acc"].append(test_dice_acc)
        results["test_miou_acc_ibg"].append(test_miou_acc_ibg)
        results["test_iou_background"].append(test_iou_background)

        results["learning_rate"].append(current_lr)


        # Save model continuously :

        file_checkpoint_exp = 'checkpoint_random_v' + str(hyperparameters.VERSION_EXPERIMENT) + '_{}'
        file_checkpoint = file_checkpoint_exp.format(epoch+1)

        if(test_miou_acc_ibg >= 0.27 or test_miou_acc_all >= 0.48 ):
            save_training_checkpoint({
                'epoch': epoch+1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }, file_checkpoint,
            )


        # Update results_line dictionary

        results_line["epoch"] = epoch+1

        results_line["train_loss"] = train_loss
        results_line["train_acc"] = train_acc
        results_line["train_miou_acc_all"] = train_miou_acc_all
        results_line["train_dice_acc"] = train_dice_acc
        results_line["train_miou_acc_ibg"] = train_miou_acc_ibg
        results_line["train_iou_background"] = train_iou_background

        results_line["test_loss"] = test_loss
        results_line["test_acc"] = test_acc
        results_line["test_miou_acc_all"] = test_miou_acc_all
        results_line["test_dice_acc"] = test_dice_acc
        results_line["test_miou_acc_ibg"] = test_miou_acc_ibg
        results_line["test_iou_background"] = test_iou_background

        results_line["learning_rate"] = current_lr

        # Save data results in a file
        save_results_in_file(results_line, hyperparameters)


    # 6. Return the filled results at the end of the epochs
    return results


In [ ]:
import torchvision.models as models
from torchvision.models.segmentation.deeplabv3 import DeepLabHead

#!pip install focal_loss_torch
#from focal_loss.focal_loss import FocalLoss

!pip install segmentation-models-pytorch
import segmentation_models_pytorch as smp

from torch.optim.lr_scheduler import StepLR


# Botorch bayesian optimization

!pip install botorch



import botorch
from botorch.models import SingleTaskGP
from botorch.utils import standardize
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition import ExpectedImprovement
from botorch.sampling import SobolQMCNormalSampler
from botorch.acquisition import qNoisyExpectedImprovement
from botorch.optim import optimize_acqf

from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qExpectedImprovement


"""
HYPERPARAMETER SETTINGS
"""


class Hyperparameters:
    def __init__(self, version_experiment, optimizer, architecture_used, network, batch_size, learning_rate, momentum, weight_decay, loss_function, num_epochs):
        self.VERSION_EXPERIMENT = version_experiment
        self.OPTIMIZER = optimizer
        self.ARCHITECTURE_USED = architecture_used
        self.NETWORK = network
        self.BATCH_SIZE = batch_size
        self.LEARNING_RATE = learning_rate
        self.MOMENTUM = momentum
        self.WEIGHT_DECAY = weight_decay
        self.LOSS_FUNCTION = loss_function
        self.NUM_EPOCHS = num_epochs



def process_experiment(experiment):
    # Convert keys to lowercase
    experiment = {k.lower(): v for k, v in experiment.items()}
    return experiment


def extractVersionFromRSFileName(filename_random_search):

    # Find the position of the underscore before the version
    underscore_index = filename_random_search.rfind("v")

    # Extract the version substring
    version = filename_random_search[underscore_index + 1 : -4]

    return version


"""
    Reverse mapping function, takes the numerical value as well as the hyperparameter concerned in input,
    and returns the original value
"""
def reverse_mapping(hyperparameter, value_to_reverse):
    features_table = original_bound_values[hyperparameter]
    # Create a dictionary to map batch sizes to integer values
    size_mapping = {size: idx for idx, size in enumerate(features_table)}

    # Reverse the key-value pairs in size_mapping
    reversed_size_mapping = {idx: size for size, idx in size_mapping.items()}

    value_to_reverse = round(value_to_reverse)
    return reversed_size_mapping[value_to_reverse]



"""
From here, the data can be reloaded and variables destroyed to carry out multiple runs
"""
FIRST_RUN = True

# Number of experiments
num_experiments = 50

# start index
start_index = 2044

"""
LOOP STARTING
"""
for rs_experiment in range(start_index, start_index + num_experiments):



  data_path = Path("/content/gdrive/MyDrive/Thesis_deeplearning_models/")
  filename_hyperparameters_experiments = data_path / 'hyperparameters_versions.txt'

  # Read the file starting at experiment 1000

  hyperparameters_experiments = []

  # Read the contents of the file
  with open(filename_hyperparameters_experiments, 'r') as file:
    content = file.read()


  content = "[" + content + "]"

  # Parse the content as JSON
  hyperparameters_experiments = json.loads(content)

  # Filter experiments with version below 1000
  hyperparameters_experiments = [hyperparameters_experiment for hyperparameters_experiment in hyperparameters_experiments if hyperparameters_experiment["VERSION_EXPERIMENT"] >= 1000]

  # Transform keys to lowercase and remove specified keys
  filtered_hyperparameters_experiments = []
  for hyperparameters in hyperparameters_experiments:
      filtered_hyperparameters = {key.lower(): value for key, value in hyperparameters.items() if key not in ["NUM_EPOCHS"]}
      filtered_hyperparameters_experiments.append(filtered_hyperparameters)



  # Remove first duplicates
  seen = set()
  result = []
  for filtered_hyperparameters in filtered_hyperparameters_experiments:
      if filtered_hyperparameters["version_experiment"] not in seen:
          seen.add(filtered_hyperparameters["version_experiment"])
          result.append(filtered_hyperparameters)


  filtered_hyperparameters_experiments = result





  """

  """


  # We retrieve all the txt files in the folder
  file_list = []

  for root, dirs, files in os.walk(data_path):
      for file in files:
          if file.startswith('results_randomsearch') or file.startswith('results_bo'):
              file_path = os.path.join(root, file)
              file_list.append(file_path)

  print(file_list)


  experiments = []

  # Loop through the files

  for file_path in file_list:

      adapted_filename = "modified_data_v" + extractVersionFromRSFileName(file_path) + ".txt"
      test_post = data_path / adapted_filename

      # Read the text file
      with open(file_path, 'r') as file:
          content = file.read()

      # Add "[" at the beginning and "]" at the end
      content = '[' + content + ']'

      # Add delimiter "," between each symbol "}{"
      content = content.replace('}\n{', '},{')

      # Save the modified content to a new text file
      with open(test_post, 'w') as file:
          file.write(content)



      """
          read modified file && update the table
      """


      with open(test_post, 'r') as file:
          content_transformed = file.read()

      data_experiment = json.loads(content_transformed)

      for epoch in data_experiment:
          experiments.append(
            {
                "version_experiment": int(extractVersionFromRSFileName(file_path)),
                "epoch": epoch["epoch"],
                "test_miou_acc_ibg": epoch["test_miou_acc_ibg"],
                "test_miou_acc_all": epoch["test_miou_acc_all"]

            }
          )



  """
      Combining the 2 arrays to prepare the data for bayesian optimization
  """

  for filtered_hyperparameters in filtered_hyperparameters_experiments:
      current_version = filtered_hyperparameters["version_experiment"]

      current_optimizer = filtered_hyperparameters["optimizer"]
      current_architecture_used = filtered_hyperparameters["architecture_used"]
      current_network = filtered_hyperparameters["network"]
      current_batch_size = filtered_hyperparameters["batch_size"]
      current_learning_rate = filtered_hyperparameters["learning_rate"]
      current_momentum = filtered_hyperparameters["momentum"]
      current_weight_decay = filtered_hyperparameters["weight_decay"]
      current_loss_function = filtered_hyperparameters["loss_function"]


      for experiment in experiments:

          if current_version == experiment["version_experiment"]:


              experiment["optimizer"] = current_optimizer
              experiment["architecture_used"] = current_architecture_used
              experiment["network"] = current_network
              experiment["batch_size"] = current_batch_size
              experiment["learning_rate"] = current_learning_rate
              experiment["momentum"] = current_momentum
              experiment["weight_decay"] = current_weight_decay
              experiment["loss_function"] = current_loss_function




  count = 0
  for exp in experiments:
      count = count + 1



  print("number of total rows " + str(count))



  # Printing data in txt_file
  """
    Posting the table into a txt file
  """

  post_training_data_file = data_path / "training_data_opti.txt"

  with open(post_training_data_file, 'w') as file:
      # Write the experiments to the file using json.dump
      json.dump(experiments, file)
      file.write('\n')





  """
    TEST running bayesian opti part
  """




  data_path_bayesian_opti = Path("/content/gdrive/MyDrive/Thesis_deeplearning_models/")

  filename_training_bayesian_opti = data_path_bayesian_opti / "training_data_opti.txt"

  # Read the contents of the file
  with open(filename_training_bayesian_opti, 'r') as file:
      content = file.read()

  # Parse the content as JSON
  training_data_bayesian_opti = json.loads(content)

  import numpy as np




  original_bound_values = {
      'optimizer':['SGD', 'Adam'],
      'architecture_used': ['DeepLabV3'],
      'network': ['resnet18'],
      'batch_size': [4,8,16],
      'momentum': [0.9],
      'loss_function': ['CrossEntropyLoss']

  }


  transformed_bound_values = {}
  training_data_bayesian_opti_modified = training_data_bayesian_opti.copy()
  valid_rows = []


  """
      Transform the original values into values that can be processed by the model
  """
  for row_training_data_modified in training_data_bayesian_opti_modified:
      ok = True
      for key in original_bound_values:
          features_table = original_bound_values[key]
          # Create a dictionary to map batch sizes to integer values
          size_mapping = {size: idx for idx, size in enumerate(features_table)}

          # Perform integer encoding on the dataset
          encoded_data = [size_mapping[features] for features in features_table]
          tuple_encoded_data = (encoded_data[0], encoded_data[-1])
          # Updates the values to numerical ones
          transformed_bound_values[key] = tuple_encoded_data

          if row_training_data_modified[key] in size_mapping:
              row_training_data_modified[key] = size_mapping[row_training_data_modified[key]]
          else:
              ok = False


      if(ok == True):
          valid_rows.append(row_training_data_modified)


  training_data_bayesian_opti_modified = valid_rows



  countb = 0
  for features_row_modified in training_data_bayesian_opti_modified:
  #   print(features_row_modified)
      countb+=1
  print("Number of valid rows : " + str(countb))


  # Define the bounds for each hyperparameter
  bounds = [
      (1,30),                                             # What is the best epoch between 1 and 30? We ran experiments between 10 and 30.
      transformed_bound_values['optimizer'],              # optimizer: 0 for 'SGD', 1 for 'Adam'
      transformed_bound_values['architecture_used'],      # architecture_used: 0 for 'DeepLabV3'
      transformed_bound_values['network'],                # network: 0 for 'resnet18'
      transformed_bound_values['batch_size'],             # batch_size: 0 for 4, 1 for 8, 2 for 16
      (0.0001, 0.9),                                      # learning_rate
      transformed_bound_values['momentum'],               # momentum: 0.9 (fixed value)
      (0.0001, 0.9),                                      # weight_decay
      transformed_bound_values['loss_function'],          # loss_function: 0 for 'CrossEntropyLoss', 1 for 'Focal Loss', 2 'Dice Loss'
  ]



  X = torch.tensor([[features_row['epoch'], features_row['optimizer'], features_row['architecture_used'],
      features_row['network'], features_row['batch_size'], features_row['learning_rate'],
      features_row['momentum'], features_row['weight_decay'], features_row['loss_function']]
      for features_row in training_data_bayesian_opti_modified], dtype=torch.double)



  y_ibg = torch.tensor([[features_row['test_miou_acc_all']] for features_row in training_data_bayesian_opti_modified])

  X = torch.tensor(X, dtype=torch.double)
  y_ibg = torch.tensor(y_ibg, dtype=torch.double)

  best_observed_y_ibg = y_ibg.max().item()
  print("best observed value for y_ibg " + str(best_observed_y_ibg))

  """

  """

  bounds = np.array(bounds)
  # Reshape the bounds to a 2 x 9 tensor
  bounds = bounds.T
  # Convert the bounds NumPy array to a PyTorch tensor
  bounds = torch.tensor(bounds)

  # SingleTaskGP model
  model_ibg = SingleTaskGP(X, y_ibg)

  # Define the acquisition function to maximize (e.g., Upper Confidence Bound)
  #acq_func = UpperConfidenceBound(model_ibg, beta=0.1)


  mll = ExactMarginalLogLikelihood(model_ibg.likelihood,model_ibg)
  fit_gpytorch_model(mll)

  acq_func = qExpectedImprovement(model_ibg, best_f=best_observed_y_ibg)


  # Optimize the hyperparameters based on the defined acquisition function
  best_hyperparameters_ibg, _ = optimize_acqf(
      acq_function=acq_func,
      bounds=bounds,
      q=1,
      num_restarts=1000,
      raw_samples=1024,
  )



  # Change the tensor to a list
  best_operational_hyperparameters = best_hyperparameters_ibg.tolist()[0]



  best_operational_hyperparameters = {
      'epoch': round(best_operational_hyperparameters[0]),
      'optimizer': reverse_mapping("optimizer",best_operational_hyperparameters[1]),
      'architecture_used': reverse_mapping("architecture_used",best_operational_hyperparameters[2]),
      'network': reverse_mapping("network",best_operational_hyperparameters[3]),
      'batch_size': reverse_mapping("batch_size",best_operational_hyperparameters[4]),
      'learning_rate': best_operational_hyperparameters[5],
      'momentum': reverse_mapping("momentum",best_operational_hyperparameters[6]),
      'weight_decay': best_operational_hyperparameters[7],
      'loss_function': reverse_mapping("loss_function",best_operational_hyperparameters[8])

}

  print(best_operational_hyperparameters)



  """
    Pre-training transformation
  """
  hyperparams = best_operational_hyperparameters
  hyperparams['num_epochs'] = hyperparams['epoch']


  """
    Training run
  """

  hyperparameters = Hyperparameters(
      rs_experiment,
      hyperparams['optimizer'],
      hyperparams['architecture_used'],
      hyperparams['network'],
      hyperparams['batch_size'],
      hyperparams['learning_rate'],
      hyperparams['momentum'],
      hyperparams['weight_decay'],
      hyperparams['loss_function'],
      hyperparams['num_epochs'],

  )

  print("Version", rs_experiment)
  print("Optimizer", hyperparams['optimizer'])
  print("Architecture", hyperparams['architecture_used'])
  print("Network", hyperparams['network'])
  print("Batch size", hyperparams['batch_size'])
  print("Learning Rate", hyperparams['learning_rate'])
  print("Momentum", hyperparams['momentum'])
  print("Weight Decay", hyperparams['weight_decay'])
  print("Loss function", hyperparams['loss_function'])
  print("Num epochs", hyperparams['num_epochs'])


  # We create a train_loader by passing the train dataset
  train_loader = torch.utils.data.DataLoader(dataset = train_1,
                                            batch_size = hyperparameters.BATCH_SIZE,
                                            shuffle = True,
                                            drop_last=True) #shuffle ensures we traverse images in different order across epochs

  # We create a test loader by passing the test dataset
  test_loader = torch.utils.data.DataLoader(dataset = test_1,
                                            batch_size = hyperparameters.BATCH_SIZE,
                                            shuffle = False,
                                            drop_last=True) #we don't do shuffle here because we only do 1 forward pass



  # Setup loss
  if(hyperparameters.LOSS_FUNCTION == "CrossEntropyLoss"):
      print("CrossEntropyLoss")
      loss_fn = nn.CrossEntropyLoss()

      if(hyperparameters.ARCHITECTURE_USED == "DeepLabV3"):
          print("DeepLabV3 no activation")
          model = smp.DeepLabV3(encoder_name=hyperparameters.NETWORK,
            encoder_depth=5,
            encoder_weights='imagenet',
            decoder_channels=256,
            in_channels=3,
            classes=104,
            activation=None,
            upsampling=8,
            aux_params=None)

      if(hyperparameters.ARCHITECTURE_USED == "Unet"):
          print("Unet no activation")
          model = smp.Unet(encoder_name=hyperparameters.NETWORK,
            encoder_depth=5,
            encoder_weights='imagenet',
            decoder_use_batchnorm=True,
            decoder_channels=(256, 128, 64, 32, 16),
            decoder_attention_type=None,
            in_channels=3,
            classes=104,
            activation=None,
            aux_params=None)


  if(hyperparameters.LOSS_FUNCTION == "FocalLoss"):
      print("FocalLoss")
      loss_fn = smp.losses.FocalLoss(mode='multiclass', gamma=0.7)

      if(hyperparameters.ARCHITECTURE_USED == "DeepLabV3"):
          print("DeepLabV3 softmax activation")
          model = smp.DeepLabV3(encoder_name=hyperparameters.NETWORK,
            encoder_depth=5,
            encoder_weights='imagenet',
            decoder_channels=256,
            in_channels=3,
            classes=104,
            activation='softmax',
            upsampling=8,
            aux_params=None)

      if(hyperparameters.ARCHITECTURE_USED == "Unet"):
          print("Unet softmax activation")
          model = smp.Unet(encoder_name=hyperparameters.NETWORK,
            encoder_depth=5,
            encoder_weights='imagenet',
            decoder_use_batchnorm=True,
            decoder_channels=(256, 128, 64, 32, 16),
            decoder_attention_type=None,
            in_channels=3,
            classes=104,
            activation='softmax',
            aux_params=None)


  if(hyperparameters.LOSS_FUNCTION == "DiceLoss"):
      print("DiceLoss")
      loss_fn = smp.losses.DiceLoss(mode='multiclass', classes=list(range(1, 104)),
                              log_loss=False,
                              from_logits=True,
                              smooth=0.0,
                              ignore_index=None,
                              eps=1e-07)


      if(hyperparameters.ARCHITECTURE_USED == "DeepLabV3"):
          print("DeepLabV3 softmax activation")
          model = smp.DeepLabV3(encoder_name=hyperparameters.NETWORK,
            encoder_depth=5,
            encoder_weights='imagenet',
            decoder_channels=256,
            in_channels=3,
            classes=104,
            activation='softmax',
            upsampling=8,
            aux_params=None)

      if(hyperparameters.ARCHITECTURE_USED == "Unet"):
          print("Unet softmax activation")
          model = smp.Unet(encoder_name=hyperparameters.NETWORK,
            encoder_depth=5,
            encoder_weights='imagenet',
            decoder_use_batchnorm=True,
            decoder_channels=(256, 128, 64, 32, 16),
            decoder_attention_type=None,
            in_channels=3,
            classes=104,
            activation='softmax',
            aux_params=None)




  # Setup optimizer
  # SGD
  if(hyperparameters.OPTIMIZER == "SGD"):

      """if(hyperparameters.MOMENTUM == None and hyperparameters.WEIGHT_DECAY != None):
          print("sgd momentum none")
          optimizer = torch.optim.SGD(model.parameters(), lr=hyperparameters.LEARNING_RATE,
                                                         weight_decay=hyperparameters.WEIGHT_DECAY)

      if(hyperparameters.WEIGHT_DECAY == None and hyperparameters.MOMENTUM != None):
          print("sgd weight decay none")
          optimizer = torch.optim.SGD(model.parameters(), lr=hyperparameters.LEARNING_RATE,
                                                        momentum=hyperparameters.MOMENTUM)

      if(hyperparameters.MOMENTUM == None and hyperparameters.WEIGHT_DECAY == None):
          print("sgd momentum none - weight decay none")
          optimizer = torch.optim.SGD(model.parameters(), lr=hyperparameters.LEARNING_RATE)

      if(hyperparameters.MOMENTUM != None and hyperparameters.WEIGHT_DECAY != None):
      """
      optimizer = torch.optim.SGD(model.parameters(), lr=hyperparameters.LEARNING_RATE,
                                                          momentum=hyperparameters.MOMENTUM,
                                                          weight_decay=hyperparameters.WEIGHT_DECAY)


  # ADAM
  if(hyperparameters.OPTIMIZER == "Adam"):

      """if(hyperparameters.MOMENTUM == None and hyperparameters.WEIGHT_DECAY != None):
          print("adam momentum none")
          optimizer = torch.optim.Adam(model.parameters(), lr=hyperparameters.LEARNING_RATE,
                                                          betas=(None,0.999),
                                                          weight_decay=hyperparameters.WEIGHT_DECAY)



      if(hyperparameters.WEIGHT_DECAY == None and hyperparameters.MOMENTUM != None):
          print("adam weight decay none")
          optimizer = torch.optim.Adam(model.parameters(), lr=hyperparameters.LEARNING_RATE,
                                                          betas=(hyperparameters.MOMENTUM,0.999)
                                                          )


      if(hyperparameters.MOMENTUM == None and hyperparameters.WEIGHT_DECAY == None):
          print("adam momentum none - weight decay none")
          optimizer = torch.optim.Adam(model.parameters(), lr=hyperparameters.LEARNING_RATE,
                                                          betas=(None,0.999))

      if(hyperparameters.MOMENTUM != None and hyperparameters.WEIGHT_DECAY != None):
      """
      print("adam momentum - weight decay ")
      optimizer = torch.optim.Adam(model.parameters(), lr=hyperparameters.LEARNING_RATE,
                                                          betas=(hyperparameters.MOMENTUM,0.999),
                                                          weight_decay=hyperparameters.WEIGHT_DECAY)




  #optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
  #scheduler = StepLR(optimizer, step_size=1, gamma=1)


  # Pass the model to the assigned device
  model = model.to(device)


  # Start the timer
  start_time = timer()

  model_results = train(model=model,
                          train_dataloader=train_loader,
                          test_dataloader=test_loader,
                          optimizer=optimizer,
                          loss_fn=loss_fn,
                          epochs=hyperparameters.NUM_EPOCHS,
                          hyperparameters=hyperparameters)
  end_time = timer()
  print(f"Total training time: {end_time-start_time:.3f} seconds")

  FIRST_RUN = False

  if not FIRST_RUN:
      del train_loader
      del test_loader
      del model
      del loss_fn
      del optimizer
      del model_results

      # test running baye opti
      del(X)
      del(y_ibg)
      del(model_ibg)
      del(acq_func)
      del(best_hyperparameters_ibg)
      del(best_operational_hyperparameters)





#plot_loss_curves(model_results)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=826a74a97a9980f80f44f05ec2c3d276f8ea2a5e13cc748fd54abdb0d6f523f0
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=4138f09f06e9a07a6213c71496a1a9d2ee6d9f6650bc7dfb271f2d1458be9715
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c6

<ipython-input-8-e0fd885e9dff>:376: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.double)
<ipython-input-8-e0fd885e9dff>:377: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_ibg = torch.tensor(y_ibg, dtype=torch.double)
/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, 

{'epoch': 20, 'optimizer': 'Adam', 'architecture_used': 'DeepLabV3', 'network': 'resnet18', 'batch_size': 8, 'learning_rate': 0.9, 'momentum': 0.9, 'weight_decay': 0.9, 'loss_function': 'CrossEntropyLoss'}
Version 2044
Optimizer Adam
Architecture DeepLabV3
Network resnet18
Batch size 8
Learning Rate 0.9
Momentum 0.9
Weight Decay 0.9
Loss function CrossEntropyLoss
Num epochs 20
CrossEntropyLoss
DeepLabV3 no activation


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 268MB/s]


adam momentum - weight decay 


...

Epoch: 22 | train_loss: 0.4542 | train_acc: 0.8774 | train_miou_acc_all: 0.7836 | train_dice_acc: 0.8774 | train_miou_acc_ibg: 0.4193 | train_iou_background: 0.3643 | test_loss: 1.4661 | test_acc: 0.6333 | test_miou_acc_all: 0.4653 | test_dice_acc: 0.6333 | test_miou_acc_ibg: 0.2359 | test_iou_background: 0.2294 | learning_rate: 0.0027 | 


100%|██████████| 23/23 [14:00<00:00, 36.56s/it]

Epoch: 23 | train_loss: 0.4341 | train_acc: 0.8822 | train_miou_acc_all: 0.7914 | train_dice_acc: 0.8822 | train_miou_acc_ibg: 0.4365 | train_iou_background: 0.3549 | test_loss: 1.4829 | test_acc: 0.6312 | test_miou_acc_all: 0.4631 | test_dice_acc: 0.6312 | test_miou_acc_ibg: 0.2386 | test_iou_background: 0.2245 | learning_rate: 0.0027 | 
Total training time: 840.941 seconds
['/content/gdrive/MyDrive/Thesis_deeplearning_models/results_bo_v2016.txt', '/content/gdrive/MyDrive/Thesis_deeplearning_models/results_bo_v2017.txt', '/content/gdrive/MyDrive/Thesis_deeplearning_models/results_bo_v2018.txt', '/content/gdrive/MyDrive/Thesis_deeplearning_models/results_bo_v2019.txt', '/content/gdrive/MyDrive/Thesis_deeplearning_models/results_bo_v2020.txt', '/content/gdrive/MyDrive/Thesis_deeplearning_models/results_bo_v2021.txt', '/content/gdrive/MyDrive/Thesis_deeplearning_models/results_bo_v2022.txt', '/content/gdrive/MyDrive/Thesis_deeplearning_models/results_bo_v2023.txt', '/content/gdrive/MyDr

number of total rows 1912
Number of valid rows : 1848
best observed value for y_ibg 0.48272496461868286


<ipython-input-8-e0fd885e9dff>:376: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.double)
<ipython-input-8-e0fd885e9dff>:377: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_ibg = torch.tensor(y_ibg, dtype=torch.double)
/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, 

In [ ]:
import random

# Load the pre-trained segmentation model
state_dict = torch.load('/content/gdrive/MyDrive/Thesis_deeplearning_models/deeplabv3_resnet101_v30_6.pth', map_location=torch.device('cpu') )

# Set the model to evaluation mode
#model = model
state_dict_real = state_dict['state_dict']
model.load_state_dict(state_dict_real)
model.eval()

# loader
test_X, test_y = next(iter(test_loader))
test_image = test_X[1]
ground_truth = test_y[1]

print("test image shape", test_image.shape)
print("ground_truth shape ", ground_truth.shape)

# Prepare to pass to the model
test_image = test_image.unsqueeze(dim=0)
print("timage", test_image.shape)

# Convert the image to a PyTorch tensor
img_tensor = test_image.cuda()
#img_tensor = test_image

# Predict the segmentation mask
with torch.no_grad():
    output = model.forward(img_tensor)

print("output shape", output.shape)

# Remove the batch size dimension
prediction = output.squeeze()
# Apply softmax on output logits
#prediction = F.softmax(prediction, dim=0)
# Classes dimension becomes first so dim=0
max_values = torch.argmax(prediction, dim=0)
prediction = max_values


# Prediction gives a 256*256 output
print("prediction", prediction.shape)

# Pass the prediction to the cpu
prediction = prediction.cpu()
print(prediction)
#-----

# Preparation test image and ground_truth
test_image = test_image.squeeze()
#test_image = test_image.permute(1, 2, 0)
print("test_image+", test_image.shape)
ground_truth = ground_truth.squeeze()

ground_truth = 255 * ground_truth
ground_truth = ground_truth.long()

print(ground_truth)

"""
--------------- METRICS --------------
"""

tp, fp, fn, tn = smp.metrics.get_stats(prediction, ground_truth, mode='multiclass', num_classes=104)
iiou_acc = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro").item()
ddice_acc = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro").item()
aacc = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro").item()



# Calculate pixel accuracy
print("SMP Pixel Accuracy", aacc)
print("SMP Intersection over union", iiou_acc)
print("SMP Dice coefficient metric", ddice_acc)

# Calculate pixel accuracy
print("Pixel Accuracy", calculate_pixel_accuracy(ground_truth, prediction))
print("Intersection over union", calculate_iou(ground_truth, prediction))
print("Dice coefficient metric", dice_coefficient(ground_truth, prediction))


img_unnorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

test_image = img_unnorm(test_image)

test_image = test_image.permute(1, 2, 0)

# normalize the tensor values to the range [0, 255]
test_image = test_image * 255.0

# convert the tensor to a numpy array
test_image = test_image.clamp(0, 255).byte().numpy()

## PLOT

plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_image)
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth)
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(prediction)
plt.show()



test image shape torch.Size([3, 224, 224])
ground_truth shape  torch.Size([1, 224, 224])
timage torch.Size([1, 3, 224, 224])
output shape torch.Size([1, 104, 224, 224])
prediction torch.Size([224, 224])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
test_image+ torch.Size([3, 224, 224])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
SMP Pixel Accuracy 0.9833673238754272
SMP Intersection over union 0.07244611531496048
SMP Dice coefficient metric 0.13510443270206451
Pixel Accuracy 13.510443239795919
Intersection over union 0.0337761080994898
Dice coefficient metric 0.0
